# Importing Processed Data into PostgreSQL

## 📌 Overview
This notebook connects to a **PostgreSQL database** and imports:
- **Word frequency statistics** (term occurrences in documents).
- **Word co-occurrence statistics** (relationships between terms).
- **Updates** calculations within **PostgreSQL**.

In [1]:
# Import core libraries
import os
import sys
import pandas as pd

# Import custom functions
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..", "src")))

# Import database handling functions and visualization functions
from database import get_database_engine, process_and_import_data

# Move the working directory one level up 
os.chdir(os.path.abspath(os.path.join(os.getcwd(), "..")))

## 1. Connect to the PostgreSQL Database

We retrieve database credentials from the `.env` file and establish a **secure connection**.  You will need to provide your own credentials for your own database.

In [2]:
# Establish Database Connection
engine = get_database_engine()

# Test connection
try:
    with engine.connect() as connection:
        print("✅ Successfully connected to the database!")
except Exception as e:
    print(f"❌ Connection failed: {e}")


✅ Successfully connected to the database!


## 2. Define Data Import Parameters

We will:
- Scan **each term folder** in `data`
- **Skip already processed terms** (unless `skip_processed=False`).


In [4]:
# Define Root Folder and Import Parameters

root_folder = r"D:\Msc\data"
total_docs = 16023367  # This number is the total number of documents in xDD at the time of the search.
skip_processed = True  # Set to False to reprocess all terms

## 3. Execute Data Import

This step will:
- **Check for existing processed terms** to avoid duplication.
- **Insert term counts** into `term_counts` table.
- **Insert co-occurrence statistics** into `term_cooccurrence` table.
- **Run a stored procedure** to update statistics.


In [5]:
# Run the Data Import Function

process_and_import_data(root_folder, total_docs, engine, skip_processed)


Importing terms:  77%|███████████████████████████████████████████████▌              | 841/1095 [07:11<02:05,  2.02it/s]

Inserted or updated word_counts data for term: sask craton

🌀 COPY attempt 1 — 1027 rows
📤 Trying COPY insert (attempt 1)...
✅ COPY insert completed successfully.


Importing terms: 100%|█████████████████████████████████████████████████████████████| 1095/1095 [09:25<00:00,  1.94it/s]


## 4. Verify Data in PostgreSQL

Once the import is complete, we verify the data by querying the database.


In [7]:
# 10. Query Co-Occurrence Table

query = "SELECT * FROM term_cooccurrence ORDER BY count DESC LIMIT 10"

with engine.connect() as connection:
    df_cooc = pd.read_sql(query, connection)

print("📊 Top 10 Co-Occurring Word Pairs:")
display(df_cooc)


📊 Top 10 Co-Occurring Word Pairs:


,word_1,word_2,count,prob_w1w2,mutual_information,entropy_w1w2,prob_notw1w2,entropy_notw1w2
0,new,york,12459156,0.000180,None,None,None,None
1,high,middle,6684146,0.000096,None,None,None,None
2,present,study,5894209,0.000085,None,None,None,None
3,age,year,5615331,0.000081,None,None,None,None
4,related,article,5429790,0.000078,None,None,None,None
5,one,two,4990364,0.000072,None,None,None,None
6,data,analysis,4711650,0.000068,None,None,None,None
7,related,view,4395150,0.000063,None,None,None,None
8,figure,shown,4340280,0.000063,None,None,None,None
9,active,cell,4318821,0.000062,None,None,None,None


## 5. Next Steps

Now that we have:  
✔ Extracted data (`1_xdd_data_extraction.ipynb`)  
✔ Processed text statistics (`2_calculate_statistics.ipynb`)  
✔ Imported data into PostgreSQL (`3_import_to_postgres.ipynb`)  

📌 **Visualize in the Web App**

To explore semantic relationships and co-occurrence networks interactively, visit the GeoNLP web app.

➡️ Open the app in your browser: https://www.terra-datasystems.com  
➡️ Use the search box to enter geological terms (e.g., `"volcanic arc"`)  
➡️ Adjust filters, explore connections, and download CSVs for further analysis.

---

**Coming Up:**  
- Add machine learning workflows for classification and clustering.  
- Geo-tagging and spatial relationships
- Named Entity Recognition
- Relation Extraction
- Question and Answering
- Custom LLMs
- Cosine Similarity and TF-IDF
- Topic Modeling
- Text Classification
